In [1]:
import tensorflow as tf
import pandas as pd
from transformers import AutoTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

In [2]:
HUGGING_FACE_PATH = "klue/bert-base" # KLUE_BERT, BERT # Ko-GPT
# model = TFBertForSequenceClassification.from_pretrained(HUGGING_FACE_PATH, num_labels=3, from_pt=True)
# tokenizer = AutoTokenizer.from_pretrained(HUGGING_FACE_PATH)